In [1]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split

import pandas as pd
import random
import csv
import os
import math
import numpy as np
from tqdm import tqdm

from numba import jit
import matplotlib.pyplot as plt
import seaborn as sns

pd.options.mode.chained_assignment = None

from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, f_classif

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


# Kies testing_set is False of True!!

In [ ]:
# HIER!!!!!!
testing_set = True

In [ ]:
def choose_data(testing_set):
    
    if testing_set is True:
        #total = pd.read_csv("./data/training_set_VU_DM.csv")
        traindf = pd.read_hdf("./data/corrected_price_testset.hdf")
        filename = "./data/test_clean.hdf"
        
    else:
        #total = pd.read_csv("./data/test_set_VU_DM.csv")
        traindf = pd.read_hdf("./data/corrected_price.hdf")
        filename = "./data/traindf_clean.hdf"
        
    return traindf, filename
        
traindf, filename = choose_data(testing_set)

In [6]:
def downsampling(df):
    """
    Balance classes in trainingset, based on click_bool (not booking_bool)
    """
    # Get 50% of data with importance of 5 or 1
    clicks = df[df.importance != 0].index
    randoms = np.random.choice(clicks, len(df.loc[df.importance != 0]) , replace=False)
    click_sample = df.loc[randoms]
    
    # Other 50% of the data
    not_click = df[df.importance == 0].index
    random_indices = np.random.choice(not_click, len(df.loc[df.importance == 0]), replace=False)
    not_click_sample = df.loc[random_indices]

    df_new = pd.concat([not_click_sample, click_sample], axis=0)
    
    return df_new

In [7]:
def competitors(df):
    """
    Make a new column in the dataframe (competitor_bool) for when there 
    exists a competitor and there are available rooms.
    1 is True, 0 is False.
    """

    # we say that there is no competitor with a lower price
    df["competitor_lower"] = 0 #competitor_bools
    
    # comp1rate = 1 if price is lower
    df["competitor_lower"][df["comp1_rate"] == 1] = 1
    df["competitor_lower"][df["comp2_rate"] == 1] = 1
    df["competitor_lower"][df["comp3_rate"] == 1] = 1
    df["competitor_lower"][df["comp4_rate"] == 1] = 1
    df["competitor_lower"][df["comp5_rate"] == 1] = 1
    df["competitor_lower"][df["comp6_rate"] == 1] = 1
    df["competitor_lower"][df["comp7_rate"] == 1] = 1
    df["competitor_lower"][df["comp8_rate"] == 1] = 1
    
    # we say at first there is no competitor hotel available
    df["competitor_available"] = 0
    
    # availability bool = 1 if there if the competitor and expedia are available
    df["competitor_available"][df["comp1_inv"] == 1] = 1
    df["competitor_available"][df["comp2_inv"] == 1] = 1
    df["competitor_available"][df["comp3_inv"] == 1] = 1
    df["competitor_available"][df["comp4_inv"] == 1] = 1
    df["competitor_available"][df["comp5_inv"] == 1] = 1
    df["competitor_available"][df["comp6_inv"] == 1] = 1
    df["competitor_available"][df["comp7_inv"] == 1] = 1
    df["competitor_available"][df["comp8_inv"] == 1] = 1

    return df

In [8]:
def visitor_history(df):
    """
    Add column that tells us whether someone has visited a hotel before.
    Column name = total_visited; 1 is True, 0 is False.
    """
    
    # most visitors haven't visited a hotel yet
    df["visited_before"] = 0
    
    # where there is a history field filled in, visited_before is turned into 21
    df["visited_before"][df["visitor_hist_starrating"].notna() | df["visitor_hist_adr_usd"].notna()] = 1
    
    return df

In [9]:
def price_quality(df):
    """
    Add a column of ratio price/quality to the DataFrame.
    """
    
    df["price_quality"] = None
    
    df["prop_starrating"].replace(0, 0.0001,inplace=True) # TODO
    
    df["price_quality"][df["price_usd"].notna() & df["prop_starrating"].notna() & df["prop_starrating"] != 0] = df["price_usd"] / df["prop_starrating"]

    # Replace missing values with median
    df["price_quality"].fillna((df["price_quality"].median()), inplace=True)
    
    return df 



In [10]:
def price_category(df):
    """
    Add a column of categories of price_usd and a column
    that corrected price for number of nights.
    Preprocessing of quantile cut showed that categories are:
    [(6.0889999999999995, 69.0] < (69.0, 90.0] < (90.0, 110.0] 
    < (110.0, 136.0] < (136.0, 170.077] < (170.077, 239.0] < (239.0, 554655.0]]
    
    """
    
    columnames = list(df.columns)
    columnames.extend(["avg_price_propid", "std_avg_price_propid", "amount_hotels", 
                       "avg_price_propid_after", "std_avg_price_propid_after"])
    
    df = df.reindex(columns=columnames)
    
    # copy prices (for now, at the end we will just update the price i suppose)
    df["price_correction"] = df["price_usd"]
    
    print("Made all extra columns")
    display(df[["avg_price_propid", "std_avg_price_propid", "amount_hotels", 
                       "avg_price_propid_after", "std_avg_price_propid_after", "price_correction"]])
    
    amount = 0
    
    propids = list(df.prop_id.unique())
    
    for prop_id in tqdm(propids, desc="Processing propids:"): 
        # calculate average and standard deviation
        std = df["price_usd"][df["prop_id"] == prop_id].std()
        avg = df["price_usd"][df["prop_id"] == prop_id].mean() 
        
        # count how many times this hotel appears in the dataframe
        df["amount_hotels"][df["prop_id"] == prop_id] = len(df[df["prop_id"] == prop_id])
        
        # put average and standard deviation in dataframe
        df["avg_price_propid"][df["prop_id"] == prop_id] = avg
        df["std_avg_price_propid"][df["prop_id"] == prop_id] = std
        
        # If std is high, correct for number of nights
        if std > 50:
            amount += 1
            df["price_correction"][df["prop_id"] == prop_id] = df["price_usd"][df["prop_id"] == prop_id] / df["srch_length_of_stay"][df["prop_id"] == prop_id]
        
        # for now separate columns so we can compare
        df["avg_price_propid_after"][df["prop_id"] == prop_id] = df["price_correction"][df["prop_id"] == prop_id].mean()
        df["std_avg_price_propid_after"][df["prop_id"] == prop_id] = df["price_correction"][df["prop_id"] == prop_id].std()
            
    print("Amount of properties with std > 50: ", amount)
    
    # Ik kreeg hier een error sorry
#     print("Correcting ", len(df[df["std_avg_price_propid"] > 50]), " property prices")
    
    return df

In [11]:
def process_remaining_cols(df):
    """
    Add some remaining (and interesting columns) to the dataframe.
    """
    
    # Replace missing values with median
    df["prop_brand_bool"].fillna((df["prop_brand_bool"].median()), inplace=True)
    df["random_bool"].fillna((df["random_bool"].median()), inplace=True)
    
    # Boolians
#     df["prop_location_score1"].fillna((df["prop_location_score1"].mean()), inplace=True)
    df["prop_location_score1"].fillna(0, inplace=True)
    df["prop_location_score2"].fillna(0, inplace=True)
        
    # Replace missing values with mean
#     df["promotion_flag"].fillna((df["promotion_flag"].mean()), inplace=True)
    df["promotion_flag"].fillna(0, inplace=True)
    
    return df

In [12]:
def add_score(df):
    """
    Add an importance score based on click_bool and booking_bool
    """
    
    # every hotel that is clicked on gets an importance score of 1
    df["importance"] = df["click_bool"]
    
    # every hotel that is booked gets an importance score of 5 
    df["importance"][df["booking_bool"] == 1] = 5
        
    return df

### Aparte functies aanroepen voor de kolommen die je erbij wilt

In [14]:
# traindf = competitors(traindf)
# print("cleaned competitors")

# traindf = visitor_history(traindf)
# print("cleaned visitor history")

traindf = price_quality(traindf)
print("cleaned price quality")

# traindf = price_category(traindf)
# print("cleaned price category")

traindf = process_remaining_cols(traindf)
print("did remaining columns")


# Add relevant columns    
if testing_set is False:
    
    df = traindf[["prop_id", 
                  "srch_id", 
                  "position", 
                  "price_quality", 
                  #"competitor_lower", 
                  #"competitor_available", 
                  #"visited_before", 
                  "click_bool", 
                  "booking_bool",  
                  "price_usd",
                  "promotion_flag", 
                  "prop_brand_bool", 
                  "random_bool",
                  "prop_location_score1", 
                  "prop_location_score2",
                  "avg_price_propid", 
                  "std_avg_price_propid", 
                  "amount_hotels", 
                  "avg_price_propid_after", 
                  "std_avg_price_propid_after",
                  "price_correction"]]
    
    df = add_score(df)
    print("added score")
    
    # Balance data to 50% importance score or 1 or 5 and 0 
    df = downsampling(df)
    print("Downsampled data")
    
else:
    
    # df without click_bool, booking_bool and position
    df = traindf[["prop_id", 
                  "srch_id", 
                  "price_quality", 
                  #"competitor_lower", 
                  #"competitor_available", 
                  #"visited_before", 
                  "price_usd",
                  "promotion_flag", 
                  "prop_brand_bool", 
                  "random_bool",
                  "prop_location_score1", 
                  "prop_location_score2",
                  "avg_price_propid", 
                  "std_avg_price_propid", 
                  "amount_hotels", 
                  "avg_price_propid_after", 
                  "std_avg_price_propid_after",
                  "price_correction"
                 ]]

cleaned price quality
did remaining columns


In [15]:
display(df.head())
display(df.describe())

,prop_id,srch_id,price_quality,price_usd,promotion_flag,prop_brand_bool,random_bool,prop_location_score1,prop_location_score2,avg_price_propid,std_avg_price_propid,amount_hotels,avg_price_propid_after,std_avg_price_propid_after,price_correction
0,3180,1,39.666667,119.0,0,1,0,2.94,0.0691,125.520757,11.710160,185.0,125.520757,11.710160,119.0
1,5543,1,39.333333,118.0,0,1,0,2.64,0.0843,117.532579,20.483788,252.0,117.532579,20.483788,118.0
2,14142,1,24.500000,49.0,0,1,0,2.71,0.0556,51.886600,4.464016,150.0,51.886600,4.464016,49.0
3,22393,1,47.666667,143.0,0,1,0,2.40,0.0561,133.987619,27.965448,147.0,133.987619,27.965448,143.0
4,24194,1,26.333333,79.0,0,1,0,2.94,0.2090,86.308224,14.807339,214.0,86.308224,14.807339,79.0


,prop_id,srch_id,price_quality,price_usd,promotion_flag,prop_brand_bool,random_bool,prop_location_score1,prop_location_score2,avg_price_propid,std_avg_price_propid,amount_hotels,avg_price_propid_after,std_avg_price_propid_after,price_correction
count,4.959183e+06,4.959183e+06,4.959183e+06,4.959183e+06,4.959183e+06,4.959183e+06,4.959183e+06,4.959183e+06,4.959183e+06,4.959183e+06,4.959183e+06,4.959183e+06,4.959183e+06,4.959183e+06,4.959183e+06
mean,7.008111e+04,1.666460e+05,5.237394e+04,2.293575e+02,2.159033e-01,6.339091e-01,2.969189e-01,2.879367e+00,1.018317e-01,2.293575e+02,8.630156e+02,2.127442e+02,1.720690e+02,4.883592e+02,1.720690e+02
std,4.061363e+04,9.614992e+04,3.133332e+06,1.246369e+04,4.114476e-01,4.817348e-01,4.569005e-01,1.533165e+00,1.509622e-01,1.690832e+03,1.231828e+04,3.173549e+02,8.592206e+02,6.221937e+03,6.299941e+03
min,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,7.000000e-02,0.000000e+00,1.000000e+00,7.000000e-02,0.000000e+00,0.000000e+00
25%,3.502100e+04,8.331150e+04,2.976333e+01,8.500000e+01,0.000000e+00,0.000000e+00,0.000000e+00,1.790000e+00,1.400000e-03,9.515172e+01,1.817830e+01,4.900000e+01,8.982400e+01,1.817132e+01,7.500000e+01
50%,6.960700e+04,1.670950e+05,4.000000e+01,1.222600e+02,0.000000e+00,1.000000e+00,0.000000e+00,2.770000e+00,3.560000e-02,1.333800e+02,3.784726e+01,1.110000e+02,1.238700e+02,3.766439e+01,1.100000e+02
75%,1.051790e+05,2.499660e+05,5.675000e+01,1.850000e+02,0.000000e+00,1.000000e+00,1.000000e+00,4.040000e+00,1.379000e-01,1.977817e+02,8.076247e+01,2.460000e+02,1.715808e+02,7.604886e+01,1.620000e+02
max,1.408210e+05,3.327870e+05,4.541855e+09,9.661340e+06,1.000000e+00,1.000000e+00,1.000000e+00,6.980000e+00,1.000000e+00,2.398547e+05,1.122171e+06,2.376000e+03,1.198225e+05,5.611077e+05,4.830670e+06


In [16]:
# df["price_quality"] = df.price_quality.astype(np.float32)
print(df.dtypes)
print()

if df.isnull().sum().sum() != 0:    
    print("\x1b[31mMissing values: \'\x1b[0m")
    print(df.isnull().sum())
else:
    print("\x1b[31mNo missing values!! :D \'\x1b[0m")

prop_id                         int64
srch_id                         int64
price_quality                 float64
price_usd                     float64
promotion_flag                  int64
prop_brand_bool                 int64
random_bool                     int64
prop_location_score1          float64
prop_location_score2          float64
avg_price_propid              float64
std_avg_price_propid          float64
amount_hotels                 float64
avg_price_propid_after        float64
std_avg_price_propid_after    float64
price_correction              float64
dtype: object

No missing values!! :D '


## Totale dataset zonder missende waardes

In [17]:
# save the dataframe if it does not exist yet
# if not os.path.exists(filename):
#     df.to_hdf(filename, key="df", format="table")
df.to_hdf(filename, key="df", format="table")

In [18]:
# test if it worked
reread = pd.read_hdf(filename)

In [19]:
display(reread.describe())

,prop_id,srch_id,price_quality,price_usd,promotion_flag,prop_brand_bool,random_bool,prop_location_score1,prop_location_score2,avg_price_propid,std_avg_price_propid,amount_hotels,avg_price_propid_after,std_avg_price_propid_after,price_correction
count,4.959183e+06,4.959183e+06,4.959183e+06,4.959183e+06,4.959183e+06,4.959183e+06,4.959183e+06,4.959183e+06,4.959183e+06,4.959183e+06,4.959183e+06,4.959183e+06,4.959183e+06,4.959183e+06,4.959183e+06
mean,7.008111e+04,1.666460e+05,5.237394e+04,2.293575e+02,2.159033e-01,6.339091e-01,2.969189e-01,2.879367e+00,1.018317e-01,2.293575e+02,8.630156e+02,2.127442e+02,1.720690e+02,4.883592e+02,1.720690e+02
std,4.061363e+04,9.614992e+04,3.133332e+06,1.246369e+04,4.114476e-01,4.817348e-01,4.569005e-01,1.533165e+00,1.509622e-01,1.690832e+03,1.231828e+04,3.173549e+02,8.592206e+02,6.221937e+03,6.299941e+03
min,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,7.000000e-02,0.000000e+00,1.000000e+00,7.000000e-02,0.000000e+00,0.000000e+00
25%,3.502100e+04,8.331150e+04,2.976333e+01,8.500000e+01,0.000000e+00,0.000000e+00,0.000000e+00,1.790000e+00,1.400000e-03,9.515172e+01,1.817830e+01,4.900000e+01,8.982400e+01,1.817132e+01,7.500000e+01
50%,6.960700e+04,1.670950e+05,4.000000e+01,1.222600e+02,0.000000e+00,1.000000e+00,0.000000e+00,2.770000e+00,3.560000e-02,1.333800e+02,3.784726e+01,1.110000e+02,1.238700e+02,3.766439e+01,1.100000e+02
75%,1.051790e+05,2.499660e+05,5.675000e+01,1.850000e+02,0.000000e+00,1.000000e+00,1.000000e+00,4.040000e+00,1.379000e-01,1.977817e+02,8.076247e+01,2.460000e+02,1.715808e+02,7.604886e+01,1.620000e+02
max,1.408210e+05,3.327870e+05,4.541855e+09,9.661340e+06,1.000000e+00,1.000000e+00,1.000000e+00,6.980000e+00,1.000000e+00,2.398547e+05,1.122171e+06,2.376000e+03,1.198225e+05,5.611077e+05,4.830670e+06
